<a href="https://colab.research.google.com/github/ellozam/ciencia-datos-notebooks/blob/main/proyecto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pycountry
!pip install openml
!pip install gradio
!pip install plotly

import openml
import pandas as pd
import plotly.express as px
import socket
import gradio as gr
from PIL import Image  # Importamos la biblioteca PIL para mostrar imágenes en Gradio

# Función para comprobar la conexión a Internet
def comprobar_conexion():
    try:
        # Intentar conectar con google.com
        socket.create_connection(("www.google.com", 80), timeout=5)
        return True
    except (socket.timeout, socket.gaierror):
        return False

def cargar_y_filtrar(openml_id, min_gdp, max_gdp, countries, year_start, year_end, education_levels, employment_rate):
    try:
        # Verificar si hay conexión a Internet
        if not comprobar_conexion():
            return "Error: No se puede establecer conexión a Internet. Por favor, verifica tu conexión y vuelve a intentarlo."

        # Cargar el dataset desde OpenML por ID
        try:
            dataset = openml.datasets.get_dataset(openml_id)
            X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute())
            if isinstance(X, pd.DataFrame):
                df = X.copy()
                if y is not None:
                    df['gdp_per_capita'] = y
            else:
                return f"Error: El dataset con ID {openml_id} no se cargó como un DataFrame de pandas."
        except openml.exceptions.OpenMLDataNotFoundError:
            return f"Error: No se encontró el dataset con ID {openml_id} en OpenML."
        except Exception as e:
            return f"Error al cargar el dataset {openml_id}: {e}"

        # Verificar los primeros registros y las columnas para asegurarnos de los nombres correctos
        print("Primeros registros del dataset:")
        print(df.head())
        print("Columnas del dataset:", df.columns)

        # Asegurarse de que las columnas 'country' y 'gdp_per_capita' existen
        if 'country' not in df.columns:
            return "Error: No se encuentra la columna 'country' en el dataset."
        if 'gdp_per_capita' not in df.columns:
            return "Error: No se encuentra la columna 'gdp_per_capita' en el dataset."

        # Filtrar el dataset por PIB per cápita
        filtered = df[(df['gdp_per_capita'] > min_gdp) & (df['gdp_per_capita'] < max_gdp)]

        # Verificar los primeros registros de los datos filtrados
        print("Primeros registros de los datos filtrados:")
        print(filtered.head())

        # Comprobar si hay datos filtrados después de aplicar el filtro de PIB
        if filtered.empty:
            return "No hay datos disponibles después de aplicar los filtros de PIB."

        # Limpiar nombres de países y verificar que estén en formato adecuado para Plotly
        filtered['country'] = filtered['country'].str.strip()  # Eliminar espacios en blanco

        # Verificar que los nombres de los países sean reconocibles por Plotly
        print("Países presentes en los datos filtrados:")
        print(filtered['country'].unique())

        # Asegurarse de que el PIB per cápita sea numérico
        filtered['gdp_per_capita'] = pd.to_numeric(filtered['gdp_per_capita'], errors='coerce')

        # Comprobar que no hay valores NaN en PIB per cápita
        filtered = filtered.dropna(subset=['gdp_per_capita'])

        # Mapa Coroplético de PIB por país
        fig_map = px.choropleth(
            filtered,
            locations="country",  # El nombre de la columna de países
            locationmode="country names",  # Asegurarnos de que Plotly reconozca los países por nombre
            color='gdp_per_capita',  # Valor del PIB per cápita
            hover_name="country",  # Mostrar el nombre del país al pasar el cursor
            color_continuous_scale="Viridis",  # Escala de colores
            labels={'gdp_per_capita': 'PIB per cápita'},
            title="Mapa Coroplético de PIB per cápita por País"
        )

        # Guardar el gráfico como una imagen
        nombre_archivo = "mapa_pib_per_capita.png"
        fig_map.write_image(nombre_archivo)
        print(f"El mapa se ha guardado como {nombre_archivo}")

        return Image.open(nombre_archivo)  # Devolvemos la imagen para mostrarla en Gradio

    except Exception as e:
        return f"Error general: {e}"

# Interfaz Gradio con el mapa interactivo
gr.Interface(
    fn=cargar_y_filtrar,
    inputs=[
        gr.Number(label="ID del Dataset en OpenML", value=45104),  # Cambiar el ID si es necesario
        gr.Slider(minimum=1000, maximum=50000, step=100, label="PIB mínimo", value=1000),
        gr.Slider(minimum=1000, maximum=50000, step=100, label="PIB máximo", value=50000),
        gr.Textbox(label="Países (separados por coma, ej. 'USA,Canada')", value="USA"),
        gr.Slider(minimum=1900, maximum=2025, step=1, label="Año de inicio", value=2000),
        gr.Slider(minimum=1900, maximum=2025, step=1, label="Año de fin", value=2020),
        gr.Textbox(label="Niveles educativos (separados por coma, ej. 'High School,Bachelor')", value="High School"),
        gr.Slider(minimum=0, maximum=100, step=1, label="Tasa de empleo mínima (%)", value=50)
    ],
    outputs=[gr.Image(label="Mapa de PIB per cápita")],  # Cambiamos el tipo de salida a gr.Image
    title="Exploración de Datos Económicos con Mapa",
    description="Este análisis interactivo permite explorar y visualizar datos económicos como PIB per cápita, nivel educativo y tasa de empleo en varios países."
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c1891ce8f7108f3171.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
